In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc import psd as psd_func
from pycbc.filter import sigma
from pycbc.noise import frequency_noise_from_psd
from simple_pe.waveforms import make_waveform, calculate_mode_snr

import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


lal.MSUN_SI != Msun
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new


# Generate single eccentric waveform as 'data'

In [2]:
# Generate psd
ifos = ['H1']
tlen = 32
sample_rate = 4096
f_low = 20
psds = {'H1': 'aLIGOZeroDetHighPower',
        'f_low': f_low,
        'f_high': int(sample_rate/2),
        'length': tlen,
        'delta_f': 1. / tlen
        }
psd = {}
for ifo in ifos:
    psd[ifo] = psd_func.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                     psds['f_low'])

# Generate TEOBResumS-Dali waveform
data_pars = {'ecc10sqrd': 0.04, 'chirp_mass': 24, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0, 'distance': 1500}
df = 1/tlen
flen = (tlen*sample_rate)//2 + 1
raw_data = make_waveform(data_pars, df, f_low, flen, approximant='TEOBResumS-Dali')

sig = sigma(raw_data, psd['H1'], low_frequency_cutoff=f_low, high_frequency_cutoff=psd['H1'].sample_frequencies[-1])
print(f'sigma: {sig}')

# Add gaussian noise
data = {'H1': (raw_data + frequency_noise_from_psd(psd['H1'], seed=12345)).to_timeseries()}

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sigma: 22.259255848584203


# Generate eccentric harmonics and match filter data

In [3]:
# Generate TEOBResumS-Dali waveform
harm_pars = {'ecc10sqrd': 0.04, 'chirp_mass': 24, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0, 'distance': 1}
harm_dict = make_waveform(harm_pars, df, f_low, flen, approximant='TEOBResumS-Dali-Harms', n_ecc_harms=4, n_ecc_gen=6)

# Match filter harmonics
harm_perp = {}
for key in harm_dict.keys():
    harm_perp[key] = harm_dict[key] / sigma(harm_dict[key], psd[ifos[0]], low_frequency_cutoff=f_low,
                                            high_frequency_cutoff=psd[ifos[0]].sample_frequencies[-1])
mode_SNRs, _ = calculate_mode_snr(data[ifos[0]], psd[ifos[0]], harm_perp, data[ifos[0]].sample_times[0],
                                  data[ifos[0]].sample_times[-1], f_low, harm_dict.keys(), dominant_mode=0)
for key in mode_SNRs.keys():
    print(f'rho_{key}: {np.abs(mode_SNRs[key]):.2f}, phi_{key}: {np.angle(mode_SNRs[key]):.2f}')

rho_0: 21.12, phi_0: -2.48
rho_1: 3.48, phi_1: 2.71
rho_-1: 1.49, phi_-1: -1.66
rho_2: 1.19, phi_2: 0.63
